In [ ]:
import wobbles.wobbles as w
import galpy
import numpy as np
import astropy.units as apu
import galpy.orbit
import matplotlib.pyplot as plt

### Use galpy to integrate the orbit of the satellite

In [ ]:
n_snaps = 1000 # Number of timesteps
t_orbit = -1.64 # Gyr

mwp= galpy.potential.MWPotential2014 # Define potential in which satellite orbit will be integrated
times = np.linspace(0, t_orbit, n_snaps) * apu.Gyr 
init = [283. * apu.deg, -30. * apu.deg, 26. * apu.kpc,
        -2.6 * apu.mas/apu.yr, -1.3 * apu.mas/apu.yr, 140. * apu.km/apu.s] # Initial conditions of the satellite

o_sgr = galpy.orbit.Orbit(vxvv=init,radec=True) # Initialise orbit instance
o_sgr.integrate(times, mwp) # Integrate orbit
o_sgr.plot(d2='r') # Plot galactocentric radius as a function of time

### Create the satellite potential

In [ ]:
halo_pot= galpy.potential.HernquistPotential(amp=2.*1e10*apu.M_sun,a= 3.*apu.kpc) 
stlr_pot= galpy.potential.HernquistPotential(amp=2.*0.2e9*apu.M_sun,a=0.65*apu.kpc) 

sgr_pot= halo_pot+stlr_pot
galpy.potential.turn_physical_off(sgr_pot)

### Define the vertical (1-dimensional) potential in which the disc orbits will be integrated

In [ ]:
verticalmwp= galpy.potential.toVerticalPotential(mwp,1.)

### Create the Disc object that we want to perturb
discpot= list of galpy potential objects  
times= numpy array of times at which to integrate disc orbits (if the times are different when you add satellite, the code will reintegrate disc orbits)  
zlim= phase space is evaluated for -zlim to zlim  
vlim= phase space is evaluated for -vlim to vlim  
zpt= number of points along z-axis  
vpt= number of points along vz-axis

In [ ]:
disc= w.Disc(discpot=[verticalmwp],times=o_sgr.t,zlim=2.,vlim=120.,zpt=50,vpt=50)

### Add a perturbation to the disc

If you want to add an arbitrary force, you can use add_force() instead.

**satpot**= galpy potential of the satellite  
**sat**= galpy orbit instance  
**df_prop**= [[mid-plane densities],[velocity dispersion]] in internal galpy units.  
**ftype**= 'static' or 'rotate'. Use 'rotate', it means the solar neighbourhood is orbiting the ga;actic centre  
**tdep**= boolean. Do you want the perturbation calculated at each time step? Takes a pretty long time if True.  
**method**= 'fast', 'slow', 'slowest'. When to use each  
            'fast'- grid size smaller than 30 x 30   
            'slow'- grid size larger than 30 x 30 and no time dependence  
            'slowest'- grid size larger than 30 x 30 and time dependence  
**zsun**= 'fit', 'mean' or None. 'fit' and 'mean' adjust for the location of the mid-plane (This is recommended). None does not adjust

In [ ]:
p= galpy.potential.evaluateDensities(mwp,1.,0.) # find mid-plane density of MWPotential2014
disc.add_satellite(satpot=sgr_pot,sat=o_sgr,df_prop=[[p],[20.5/220.]],ftype='rotate',
                   tdep=False,method='slow',zsun='fit')

### Plot properties of the orbit as well as asymmetry and mean vertical velocity for sample orbit

In [ ]:
MTU= galpy.util.bovy_conversion.time_in_Gyr(220.,8.)

fig,(ax1,ax2)= plt.subplots(2,1,figsize=(5,6),sharex=True)

ax1.plot(o_sgr.t*MTU,o_sgr.r(o_sgr.t)*8.)
ax2.plot(disc.t*MTU,disc.Force[0,0]*galpy.util.bovy_conversion.force_in_2piGmsolpc2(220.,8.))
ax1.set_ylabel(r'$\mathrm{r\,(kpc)}$')
ax2.set_ylabel(r'$\mathrm{F_z\,(2\pi\,G\,M_\odot\,pc^{-2})}$')
ax2.set_xlabel(r'$\mathrm{t\,(Gyr)}$')

plt.subplots_adjust(wspace=0, hspace=0)

fig,(ax1,ax2)= plt.subplots(2,1,figsize=(5,8),sharex=True)
ax1.plot(disc.zA*8.,disc.A,c='C0')
ax1.plot(-disc.zA*8.,-disc.A,c='C0',ls='--',alpha=0.7)
ax2.plot(disc.z*8.,disc.meanV*220.)
ax1.set_ylabel(r'$\mathrm{A}$')
ax2.set_ylabel(r'$\langle v_{z}\rangle\,\mathrm{(km\,s^{-1})}$')
ax2.set_xlabel(r'$\mathrm{z\,(kpc)}$')

plt.subplots_adjust(wspace=0, hspace=0)